In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
from pytrends.request import TrendReq

<h3>Google Trends Data</h3>

<h3>Flu data</h3>
<ul>
    <li>Region 1: GE, NE, VD, VS</li> 
    <li>Region 2: BE, FR, JU </li>
    <li>Region 3: AG, BL, BS, SO </li>
    <li>Region 4: LU, NW, OW, SZ, UR, ZG</li> 
    <li>Region 5: AI, AR, GL, SG, SH, TG, ZH </li>
    <li>Region 6: GR, TI</li>
</ul> 